In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
# from sklearn.ensemble import RandomForestRegressor
# from sklearn.pipeline import Pipeline
from xgboost import XGBRegressor

import math
import time
import xgboost as xgb
import gc
import pickle
from numpy import array

In [2]:
train = pd.read_csv('files/train.csv')
info = pd.read_csv('files/building_metadata.csv')
weather = pd.read_csv('files/weather_train_XGB_old.csv')
test = pd.read_csv('files/test.csv')
weather_test = pd.read_csv('files/weather_test_XGB_old.csv')

In [3]:
print(weather_test.shape)

(277243, 65)


In [4]:
# create list of building_id and meter
meter_pair = train[['building_id','meter']]
meter_pair = meter_pair.groupby(['building_id', 'meter']).size().reset_index(name='Freq')
meter_pair = meter_pair[['building_id', 'meter']].values.tolist()

In [5]:
def evalerror(preds, actual):
    preds[preds<0] = 0
    return math.sqrt(np.square(np.log(preds+1)-np.log(actual+1)).mean())

In [6]:
def learn(X_full,y):
    X_train, X_valid, y_train, y_valid = train_test_split(X_full, y, train_size=0.8, test_size=0.2, random_state=0)
    params = {
            'objective': 'reg:squarederror',
#             'tree_method': 'gpu_hist',
        }

    plst = params.items()

    print('Doing')
    dtrain = xgb.DMatrix( data = X_train.values, label = y_train.values)
    num_rounds = 300
    model = xgb.train(plst, dtrain, num_rounds)

    # predict 
    dtest = xgb.DMatrix( data = X_valid.values)
    preds = model.predict(dtest)
    preds[preds<0] = 0
    print(evalerror(y_valid,preds))
    return evalerror(preds,y_valid)


In [7]:
def learn2(X_train, X_valid, y_train, y_valid):
#     X_train, X_valid, y_train, y_valid = train_test_split(X_full, y, train_size=0.8, test_size=0.2, random_state=0)
    params = {
        'objective' : 'reg:squarederror',
        'early_stopping_rounds': 10,
        'nthread' : 4,
#         'num_rounds' : 300
        'tree_method': 'gpu_hist',
    }

    plst = params.items()
    dtrain = xgb.DMatrix( data = X_train.values, label = y_train.values)
    dtest = xgb.DMatrix( data = X_valid.values)
#     return (0,0)
    num_rounds = 120
    model = xgb.train(plst, dtrain, num_rounds)

    # predict 
    preds = model.predict(dtest)
    preds[preds<0] = 0
    score = evalerror(y_valid,preds)
    return (score,preds)


In [8]:
def learn_real(X_full, y, X_test):
    y = np.log(y+1)
    X_train, X_valid, y_train, y_valid = train_test_split(X_full, y, train_size=0.8, test_size=0.2, random_state=0)
    params = {
        'objective' : 'reg:squarederror',
        'early_stopping_rounds': 10,
        'nthread' : 4,
#         'num_rounds' : 300
        'tree_method': 'gpu_hist',
    }

    plst = params.items()
    dtrain = xgb.DMatrix( data = X_train.values, label = y_train.values)
    dtest = xgb.DMatrix( data = X_test.values)
    num_rounds = 120
    model = xgb.train(plst, dtrain, num_rounds)
    # predict 
    preds = model.predict(dtest)
    preds = np.exp(preds)-1
    preds[preds<0] = 0
    return preds


In [9]:
drop_col = [
'building_id',
'meter',
'timestamp',
'meter_reading',
'site_id',
'cloud_coverage',
# 'precip_depth_1_hr',
# 'sea_level_pressure'
]

drop_col2 = [
'timestamp',
'site_id',
'cloud_coverage',
# 'precip_depth_1_hr',
# 'sea_level_pressure'
]

In [11]:
tick = time.time()
y_validall = []
y_predsall = []
for no in range(0,1449):
#     0  1449
    train2 = train[train['building_id'] == no]
    info2 = info[info['building_id'] == no]
    site_id = int(info2['site_id'])
    weather2 = weather[weather['site_id'] == site_id]
    weather_test2 = weather_test[weather_test['site_id'] == site_id]
    for meter in range(0,4):
        train3 = train2[train2['meter'] == meter]
        if not train3.empty:
            b = pd.merge(train3, weather2, on='timestamp')
            y = b['meter_reading']
            X_full = b.drop(drop_col, axis=1)
            X_full = X_full.astype('float32')
            y = y.astype('float32')
#             for learn2
#             X_train, X_valid, y_train, y_valid = train_test_split(X_full, y, train_size=0.8, test_size=0.2, random_state=0)
#             (score,preds) = learn2(X_train, X_valid, y_train, y_valid )
#             print(no,meter,score)
#             y_validall.append(y_valid.values)
#             y_predsall.append(preds)
#             for learn_real
            weather_test3 = weather_test2.drop(drop_col2, axis=1)
            preds = learn_real(X_full, y, weather_test3)
            y_predsall.append(preds)
#             print(no,meter)
    if no % 10 == 0:
        tock = time.time()
        print('Time used {} for first {} building'.format(tock-tick, no))
# print(X_train_full)
tock = time.time()
print('Time used : {}'.format(tock-tick))
# print(y_predsall)
# for learn only
# y_validall2 = np.concatenate( y_validall, axis=0 )
# y_predsall2 = np.concatenate( y_predsall, axis=0 )
# print(evalerror(y_validall2,y_predsall2))
with open('sub_XGB_old', 'wb') as f:
    pickle.dump(y_predsall, f)

Time used 0.803102970123291 for first 0 building
Time used 10.201729774475098 for first 10 building
Time used 20.235347986221313 for first 20 building
Time used 30.297406673431396 for first 30 building
Time used 38.9379665851593 for first 40 building
Time used 48.26674509048462 for first 50 building
Time used 57.48159193992615 for first 60 building
Time used 65.35446238517761 for first 70 building
Time used 75.78488087654114 for first 80 building
Time used 84.3774003982544 for first 90 building
Time used 96.73582363128662 for first 100 building
Time used 108.35880899429321 for first 110 building
Time used 123.44222092628479 for first 120 building
Time used 137.40419673919678 for first 130 building
Time used 149.80753922462463 for first 140 building
Time used 161.4852635860443 for first 150 building
Time used 170.45729875564575 for first 160 building
Time used 188.88166069984436 for first 170 building
Time used 209.01991772651672 for first 180 building
Time used 225.9893057346344 for fi